---

_You are currently looking at **version 1.2** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 2 - Pandas Introduction
All questions are weighted the same in this assignment.
## Part 1
The following code loads the olympics dataset (olympics.csv), which was derrived from the Wikipedia entry on [All Time Olympic Games Medals](https://en.wikipedia.org/wiki/All-time_Olympic_Games_medal_table), and does some basic data cleaning. 

The columns are organized as # of Summer games, Summer medals, # of Winter games, Winter medals, total # number of games, total # of medals. Use this dataset to answer the questions below.

In [2]:
import pandas as pd

df = pd.read_csv('olympics.csv', index_col=0, skiprows=1)

for col in df.columns:
    if col[:2]=='01':
        df.rename(columns={col:'Gold'+col[4:]}, inplace=True)
    if col[:2]=='02':
        df.rename(columns={col:'Silver'+col[4:]}, inplace=True)
    if col[:2]=='03':
        df.rename(columns={col:'Bronze'+col[4:]}, inplace=True)
    if col[:1]=='№':
        df.rename(columns={col:'#'+col[1:]}, inplace=True)

names_ids = df.index.str.split('\s\(') # split the index by '('

df.index = names_ids.str[0] # the [0] element is the country name (new index) 
df['ID'] = names_ids.str[1].str[:3] # the [1] element is the abbreviation or ID (take first 3 characters from that)

df = df.drop('Totals')
df.head()

,# Summer,Gold,Silver,Bronze,Total,# Winter,Gold.1,Silver.1,Bronze.1,Total.1,# Games,Gold.2,Silver.2,Bronze.2,Combined total,ID
Afghanistan,13,0,0,2,2,0,0,0,0,0,13,0,0,2,2,AFG
Algeria,12,5,2,8,15,3,0,0,0,0,15,5,2,8,15,ALG
Argentina,23,18,24,28,70,18,0,0,0,0,41,18,24,28,70,ARG
Armenia,5,1,2,9,12,6,0,0,0,0,11,1,2,9,12,ARM
Australasia,2,3,4,5,12,0,0,0,0,0,2,3,4,5,12,ANZ


### Question 0 (Example)

What is the first country in df?

*This function should return a Series.*

In [3]:
# You should write your whole answer within the function provided. The autograder will call
# this function and compare the return value against the correct solution value
def answer_zero():
    # This function returns the row for Afghanistan, which is a Series object. The assignment
    # question description will tell you the general format the autograder is expecting
    return df.iloc[0]

# You can examine what your function returns by calling it in the cell. If you have questions
# about the assignment formats, check out the discussion forums for any FAQs
answer_zero() 

# Summer           13
Gold                0
Silver              0
Bronze              2
Total               2
# Winter            0
Gold.1              0
Silver.1            0
Bronze.1            0
Total.1             0
# Games            13
Gold.2              0
Silver.2            0
Bronze.2            2
Combined total      2
ID                AFG
Name: Afghanistan, dtype: object

### Question 1
Which country has won the most gold medals in summer games?

*This function should return a single string value.*

In [4]:
def answer_one():

    return df.T.loc['Gold'].idxmax()

answer_one()

'United States'

### Question 2
Which country had the biggest difference between their summer and winter gold medal counts?

*This function should return a single string value.*

In [5]:
def answer_two():
    Gold_diff = abs(df['Gold']-df['Gold.1'])
    return Gold_diff.idxmax()

answer_two()

'United States'

### Question 3
Which country has the biggest difference between their summer gold medal counts and winter gold medal counts relative to their total gold medal count? 

$$\frac{Summer~Gold - Winter~Gold}{Total~Gold}$$

Only include countries that have won at least 1 gold in both summer and winter.

*This function should return a single string value.*

In [6]:
def answer_three():
    df_select = df[(df['Gold'] > 0) & ( df['Gold.1'] > 0)]
    #print(df_select.head())
    Gold_diff_rel = (df_select['Gold']-df_select['Gold.1']) / (df_select['Gold'] + df_select['Gold.1'])
    
    return Gold_diff_rel.idxmax()
answer_three()

'Bulgaria'

### Question 4
Write a function that creates a Series called "Points" which is a weighted value where each gold medal (`Gold.2`) counts for 3 points, silver medals (`Silver.2`) for 2 points, and bronze medals (`Bronze.2`) for 1 point. The function should return only the column (a Series object) which you created, with the country names as indices.

*This function should return a Series named `Points` of length 146*

In [7]:
def answer_four():
    Points = pd.Series(data = (df['Gold.2']*3 ) + ( df['Silver.2'] * 2) + ( df['Bronze.2']))
   # print(type(Points))
    #print(.index)
   # print(len(Points))
   # print((Points.head()))
    return Points

answer_four()

Afghanistan                            2
Algeria                               27
Argentina                            130
Armenia                               16
Australasia                           22
Australia                            923
Austria                              569
Azerbaijan                            43
Bahamas                               24
Bahrain                                1
Barbados                               1
Belarus                              154
Belgium                              276
Bermuda                                1
Bohemia                                5
Botswana                               2
Brazil                               184
British West Indies                    2
Bulgaria                             411
Burundi                                3
Cameroon                              12
Canada                               846
Chile                                 24
China                               1120
Colombia        

## Part 2
For the next set of questions, we will be using census data from the [United States Census Bureau](http://www.census.gov). Counties are political and geographic subdivisions of states in the United States. This dataset contains population data for counties and states in the US from 2010 to 2015. [See this document](https://www2.census.gov/programs-surveys/popest/technical-documentation/file-layouts/2010-2015/co-est2015-alldata.pdf) for a description of the variable names.

The census dataset (census.csv) should be loaded as census_df. Answer questions using this as appropriate.

### Question 5
Which state has the most counties in it? (hint: consider the sumlevel key carefully! You'll need this for future questions too...)

*This function should return a single string value.*

In [8]:
census_df = pd.read_csv('census.csv')
import numpy as np
census_df.head()

    

,SUMLEV,REGION,DIVISION,STATE,COUNTY,STNAME,CTYNAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,...,RDOMESTICMIG2011,RDOMESTICMIG2012,RDOMESTICMIG2013,RDOMESTICMIG2014,RDOMESTICMIG2015,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015
0,40,3,6,1,0,Alabama,Alabama,4779736,4780127,4785161,...,0.002295,-0.193196,0.381066,0.582002,-0.467369,1.030015,0.826644,1.383282,1.724718,0.712594
1,50,3,6,1,1,Alabama,Autauga County,54571,54571,54660,...,7.242091,-2.915927,-3.012349,2.265971,-2.530799,7.606016,-2.626146,-2.722002,2.592270,-2.187333
2,50,3,6,1,3,Alabama,Baldwin County,182265,182265,183193,...,14.832960,17.647293,21.845705,19.243287,17.197872,15.844176,18.559627,22.727626,20.317142,18.293499
3,50,3,6,1,5,Alabama,Barbour County,27457,27457,27341,...,-4.728132,-2.500690,-7.056824,-3.904217,-10.543299,-4.874741,-2.758113,-7.167664,-3.978583,-10.543299
4,50,3,6,1,7,Alabama,Bibb County,22915,22919,22861,...,-5.527043,-5.068871,-6.201001,-0.177537,0.177258,-5.088389,-4.363636,-5.403729,0.754533,1.107861


In [9]:
census_df_select = census_df[(census_df['SUMLEV'] == 50)]
columns_to_keep = ['STNAME',
                   'CTYNAME',
                  'COUNTY']
df2 = census_df_select[columns_to_keep]
df2.head(20)
cnt=-1
statelist=[]
cc2=np.empty(51,dtype=int)


for group, frame in df2.groupby('STNAME'):
    cnt=cnt+1
    cc = np.count_nonzero(frame['COUNTY'])
    cc2[cnt]=cc
    statelist.append(group)

print(cc2)
#print(cc3)
print(max(cc2))
print(np.argmax(cc2))
print(statelist[np.argmax(cc2)])
g=df2.groupby('STNAME') 
g.groups.keys()

#    print('Counties in state ' + group + ' have an average population of ' + str(avg))

[ 67  29  15  75  58  64   8   3   1  67 159   5  44 102  92  99 105 120
  64  16  24  14  83  87  82 115  56  93  17  10  21  33  62 100  53  88
  77  36  67   5  46  66  95 254  29  14 133  39  55  72  23]
254
43
Texas


dict_keys(['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Delaware', 'District of Columbia', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming'])

In [10]:
def answer_five():
    census_df_select = census_df[(census_df['SUMLEV'] == 50)]
    columns_to_keep = ['STNAME','CTYNAME','COUNTY']
    df2 = census_df_select[columns_to_keep]
    cnt=-1
    statelist=[]
    cc2=np.empty(51,dtype=int)
    
    for group, frame in df2.groupby('STNAME'):
        cnt=cnt+1
        cc = np.count_nonzero(frame['COUNTY'])
        cc2[cnt]=cc
        statelist.append(group)
    
    return statelist[np.argmax(cc2)]

answer_five()

'Texas'

### Question 6
**Only looking at the three most populous counties for each state**, what are the three most populous states (in order of highest population to lowest population)? Use `CENSUS2010POP`.

*This function should return a list of string values.*

In [11]:
#1) find 3 most populus county in each state
census_df_select = census_df[(census_df['SUMLEV'] == 50)]
columns_to_keep = ['STNAME','CENSUS2010POP','COUNTY']
df3 = census_df_select[columns_to_keep].sort_values(by=['CENSUS2010POP'], ascending=False)
df3.head(10)

statelist=[]
countypopsum=[]
for group, frame in df3.groupby('STNAME'):
 #   df3=df2.sort_values(by='CENSUS2010POP', ascending=False)
    #print(group)
    #print(frame)
    statelist.append(group)
   # print(frame['CENSUS2010POP'][0:3])
   # print('****',frame['CENSUS2010POP'][0:3].sum())
    countypopsum.append(frame['CENSUS2010POP'][0:3].sum())
    
    #a = frame['CENSUS2010POP']
    #print(a)



print(pd.Series(countypopsum,index=statelist).nlargest(10))

a=pd.Series(countypopsum,index=statelist).nlargest(3)
a2=a.index.tolist()
print(a2)


 

California      15924150
Texas            8269632
Illinois         6815061
New York         6321295
Florida          5564635
Arizona          5173150
Michigan         3863924
Pennsylvania     3549228
Washington       3439809
Ohio             3245910
dtype: int64
['California', 'Texas', 'Illinois']


In [12]:
def answer_six():
    census_df_select = census_df[(census_df['SUMLEV'] == 50)]
    columns_to_keep = ['STNAME','CENSUS2010POP','COUNTY']
    df3 = census_df_select[columns_to_keep].sort_values(by=['CENSUS2010POP'], ascending=False)
    #df3.head(10)
    statelist=[]
    countypopsum=[]
    
    for group, frame in df3.groupby('STNAME'):
        statelist.append(group)
        countypopsum.append(frame['CENSUS2010POP'][0:3].sum())
    a=pd.Series(countypopsum,index=statelist).nlargest(3)
    a2=a.index.tolist()

    return a2

### Question 7
Which county has had the largest absolute change in population within the period 2010-2015? (Hint: population values are stored in columns POPESTIMATE2010 through POPESTIMATE2015, you need to consider all six columns.)

e.g. If County Population in the 5 year period is 100, 120, 80, 105, 100, 130, then its largest change in the period would be |130-80| = 50.

*This function should return a single string value.*

In [13]:
def answer_seven():
    census_df_select = census_df[(census_df['SUMLEV'] == 50)]
    columns_to_keep = ['CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013',
                       'POPESTIMATE2014','POPESTIMATE2015']
    df4 = census_df_select[columns_to_keep]
    df4['diff']=df4.max(axis=1)-df4.min(axis=1)
    df4 = df4.sort_values(by='diff', ascending=False)
    
    return df4.iloc[0,0]

In [14]:
###BEHIND THE SCENES FOR Q7

census_df_select = census_df[(census_df['SUMLEV'] == 50)]
#census_df_select.head()
columns_to_keep = ['CTYNAME','POPESTIMATE2010','POPESTIMATE2011','POPESTIMATE2012','POPESTIMATE2013',
                       'POPESTIMATE2014','POPESTIMATE2015']
df4 = census_df_select[columns_to_keep]
#df4=df4.set_index('CTYNAME')
#df3.index
#df4.head()
#df4['max']=df4.max(axis=1)
#df4['min']=df4.min(axis=1)
df4['diff']=df4.max(axis=1)-df4.min(axis=1)
df4 = df4.sort_values(by='diff', ascending=False)
df4.head(3)


/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,CTYNAME,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,POPESTIMATE2013,POPESTIMATE2014,POPESTIMATE2015,diff
2667,Harris County,4108187,4181238,4262504,4352462,4447577,4538028,429841
209,Los Angeles County,9826009,9896602,9970436,10045175,10109436,10170292,344283
106,Maricopa County,3825597,3871957,3945460,4015328,4090022,4167947,342350


In [15]:
###BEHIND THE SCENES FOR Q7

print(df4.iloc[0,0])

Harris County


df3['max']=df3.max(axis=1)

### Question 8
In this datafile, the United States is broken up into four regions using the "REGION" column. 

Create a query that finds the counties that belong to regions 1 or 2, whose name starts with 'Washington', and whose POPESTIMATE2015 was greater than their POPESTIMATE 2014.

*This function should return a 5x2 DataFrame with the columns = ['STNAME', 'CTYNAME'] and the same index ID as the census_df (sorted ascending by index).*

In [62]:
def answer_eight():
    census_df_select = census_df[(census_df['SUMLEV'] == 50)]
    columns_to_keep = ['STNAME','CTYNAME','REGION','POPESTIMATE2014','POPESTIMATE2015']
    df8_select=census_df_select[columns_to_keep]
    df8_select['diffpop']=(df8_select.POPESTIMATE2015) - (df8_select.POPESTIMATE2014)
    df9= df8_select[(df8_select.diffpop > 0) & (df8_select.REGION < 3) & df8_select.CTYNAME.str.contains('Washington')]
    columns_to_keep2 = ['STNAME','CTYNAME']
    df10=df9[columns_to_keep2]
  
    return df10

In [67]:
answer_eight()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County


In [63]:

#BEHIND THE SCENES FOR Q8
census_df_select = census_df[(census_df['SUMLEV'] == 50)]
columns_to_keep = ['STNAME','CTYNAME','REGION','POPESTIMATE2014','POPESTIMATE2015']
df8_select=census_df_select[columns_to_keep]
#df8.head()
df8_select.head()
#census_df_select2=census_df_select[(census_df_select['REGION'] == 1)]
#census_df_select2.head()

,STNAME,CTYNAME,REGION,POPESTIMATE2014,POPESTIMATE2015
1,Alabama,Autauga County,3,55290,55347
2,Alabama,Baldwin County,3,199713,203709
3,Alabama,Barbour County,3,26815,26489
4,Alabama,Bibb County,3,22549,22583
5,Alabama,Blount County,3,57658,57673


In [60]:
#BEHIND THE SCENES FOR Q8
df8_select['diffpop']=(df8_select.POPESTIMATE2015) - (df8_select.POPESTIMATE2014)
df9= df8_select[(df8_select.diffpop > 0) & (df8_select.REGION < 3) & df8_select.CTYNAME.str.contains('Washington')]
#df8_select.head()
columns_to_keep2 = ['STNAME','CTYNAME']
df10=df9[columns_to_keep2]
df10.head()

,STNAME,CTYNAME
896,Iowa,Washington County
1419,Minnesota,Washington County
2345,Pennsylvania,Washington County
2355,Rhode Island,Washington County
3163,Wisconsin,Washington County
